# External Data West Ameria(USA)

## Arizona

EO: 
- https://apps.sentinel-hub.com/eo-browser/?zoom=13&lat=38.57002&lng=-116.43468&themeId=DEFAULT-THEME&visualizationUrl=https%3A%2F%2Fservices.sentinel-hub.com%2Fogc%2Fwms%2Fbd86bcc0-f318-402b-a145-015f85b9427e&datasetId=S2L2A&fromTime=2022-12-19T00%3A00%3A00.000Z&toTime=2022-12-19T23%3A59%3A59.999Z&layerId=1_TRUE_COLOR
- https://apps.sentinel-hub.com/eo-browser/?zoom=12&lat=44.65048&lng=-123.6411&themeId=DEFAULT-THEME


In [8]:
import os
import warnings
import random
from pprint import pprint
import copy
from typing import List, Tuple
import glob
import json
import csv
# import dataclasses
from joblib import Parallel, delayed
from typing import List, Set, Dict, Any

from tqdm import tqdm
import numpy as np
import pandas as pd
from box import Box, from_file
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.optimize import minimize
from sklearn.metrics import jaccard_score
import cv2
import seaborn as sns

In [9]:
class CFG(object):
    # basic
    debug: bool = False
    folds: int  = 4
    seed: int   = 417
    eps: float  = 1e-12
    
    # data
    PATH_CSV: str  =  f'../data/021_Arizona/019_seed{seed}.csv'
    PATH_NPY_ROOT = f'../data/021_Arizona/'
    PATH_DATA = '../data/Arizona/'
    os.makedirs(PATH_DATA, exist_ok=True), os.makedirs(PATH_NPY_ROOT + 'img/', exist_ok=True)
    
# box
cfg = Box({k:v for k, v in dict(vars(CFG)).items() if '__' not in k})
    
# 乱数のシードを設定
np.random.seed(cfg.seed)
random.seed(cfg.seed)
    
pprint(cfg)

{'PATH_CSV': '../data/021_Arizona/019_seed417.csv',
 'PATH_DATA': '../data/Arizona/',
 'PATH_NPY_ROOT': '../data/021_Arizona/',
 'debug': False,
 'eps': 1e-12,
 'folds': 4,
 'seed': 417}


## CLI

In [10]:
!aws --version

aws-cli/1.27.131 Python/3.8.16 Linux/5.15.0-69-generic botocore/1.29.131


## Download
few cloud scene

In [11]:
PATHS_SENTILEL2_S3 = [
    # # mountain area 
    # # Nevata
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2023/5/18/0/",
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2023/5/13/0/",
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2023/4/28/0/",
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2023/4/18/0/",
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2023/4/13/0/", # snow
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2023/4/8/0/", # snow
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2023/3/9/0/", # snow
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2022/10/10/0/",
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2022/10/30/0/",
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2023/2/7/0/", 
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2022/11/29/0/",
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2022/11/24/0/", 
    # "s3://sentinel-s2-l2a/tiles/11/S/NC/2022/11/4/0/",
    
    # # coast line and city
    # # Oregon
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2023/5/25/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2023/2/9/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2023/1/30/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2022/12/16/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2022/11/16/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2022/10/27/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2022/10/12/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2022/8/28/0/", # ground red
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2022/8/3/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2022/7/14/0/",
    # "s3://sentinel-s2-l2a/tiles/10/T/DQ/2021/8/28/0/",
    
    # flat area
    # Arizona
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/10/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/9/25/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/9/20/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/9/10/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/8/31/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/8/26/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/8/16/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/",
]

In [12]:
for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    
    !aws s3 cp {PATH_S3} {OUTPUT_PATH} --recursive --request-payer
    # break

  0%|          | 0/11 [00:00<?, ?it/s]

11-T-PH-2022-10-20
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/R10m/AOT.jp2 to ../data/Arizona/11-T-PH-2022-10-20/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/R10m/B02.jp2 to ../data/Arizona/11-T-PH-2022-10-20/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/R20m/AOT.jp2 to ../data/Arizona/11-T-PH-2022-10-20/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/R10m/B04.jp2 to ../data/Arizona/11-T-PH-2022-10-20/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/R10m/B03.jp2 to ../data/Arizona/11-T-PH-2022-10-20/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/R10m/TCI.jp2 to ../data/Arizona/11-T-PH-2022-10-20/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/R10m/B08.jp2 to ../data/Arizona/11-T-PH-2022-10-20/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/20/0/R20m/B01.jp2 to ../data/Arizona/11-T-PH-2022-10-20/R20m/B01.jp2
downl

  9%|▉         | 1/11 [04:46<47:41, 286.20s/it]

11-T-PH-2022-10-15
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/R10m/AOT.jp2 to ../data/Arizona/11-T-PH-2022-10-15/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/R10m/B02.jp2 to ../data/Arizona/11-T-PH-2022-10-15/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/R10m/B03.jp2 to ../data/Arizona/11-T-PH-2022-10-15/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/R10m/B04.jp2 to ../data/Arizona/11-T-PH-2022-10-15/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/R20m/AOT.jp2 to ../data/Arizona/11-T-PH-2022-10-15/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/R10m/B08.jp2 to ../data/Arizona/11-T-PH-2022-10-15/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/R10m/TCI.jp2 to ../data/Arizona/11-T-PH-2022-10-15/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/10/15/0/R20m/B01.jp2 to ../data/Arizona/11-T-PH-2022-10-15/R20m/B01.jp2
downl

 18%|█▊        | 2/11 [11:42<54:23, 362.64s/it]

...

 82%|████████▏ | 9/11 [1:03:55<15:02, 451.01s/it]

11-T-PH-2022-7-22
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/R10m/AOT.jp2 to ../data/Arizona/11-T-PH-2022-7-22/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/R10m/B03.jp2 to ../data/Arizona/11-T-PH-2022-7-22/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/R10m/B02.jp2 to ../data/Arizona/11-T-PH-2022-7-22/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/R20m/AOT.jp2 to ../data/Arizona/11-T-PH-2022-7-22/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/R10m/B04.jp2 to ../data/Arizona/11-T-PH-2022-7-22/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/R10m/B08.jp2 to ../data/Arizona/11-T-PH-2022-7-22/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/R10m/TCI.jp2 to ../data/Arizona/11-T-PH-2022-7-22/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/22/0/R20m/B01.jp2 to ../data/Arizona/11-T-PH-2022-7-22/R20m/B01.jp2
download: s3://sentine

 91%|█████████ | 10/11 [1:09:57<07:03, 423.74s/it]

11-T-PH-2022-7-7
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/R10m/AOT.jp2 to ../data/Arizona/11-T-PH-2022-7-7/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/R10m/B02.jp2 to ../data/Arizona/11-T-PH-2022-7-7/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/R10m/B04.jp2 to ../data/Arizona/11-T-PH-2022-7-7/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/R20m/AOT.jp2 to ../data/Arizona/11-T-PH-2022-7-7/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/R10m/B03.jp2 to ../data/Arizona/11-T-PH-2022-7-7/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/R10m/B08.jp2 to ../data/Arizona/11-T-PH-2022-7-7/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/R10m/TCI.jp2 to ../data/Arizona/11-T-PH-2022-7-7/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/PH/2022/7/7/0/R20m/B01.jp2 to ../data/Arizona/11-T-PH-2022-7-7/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/11

100%|██████████| 11/11 [1:14:56<00:00, 408.77s/it]


## Loading data

In [13]:
BANDS = [
    'R20m/B01', 'R10m/B02', 'R10m/B03', 'R10m/B04', 'R20m/B05', 'R20m/B06', 
    'R20m/B07', 'R10m/B08', 'R20m/B8A', 'R60m/B09', 'R20m/B11', 'R20m/B12',
]

In [14]:
PATHS_NPY = []

for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    
    imgs = []
    
    for BAND in BANDS:
        PATH_BAND = f'{OUTPUT_PATH}{BAND}.jp2'
        img = cv2.imread(PATH_BAND, cv2.IMREAD_UNCHANGED)        
        if 'R20m' in BAND:
            img = cv2.resize(img, (img.shape[1]*2, img.shape[0]*2), interpolation=1)
        elif 'R60m' in BAND:
            img = cv2.resize(img, (img.shape[1]*6, img.shape[0]*6), interpolation=1)

        imgs.append(img)
        
    imgs = np.stack(imgs, axis=2)
    
    for h_idx in range(imgs.shape[0]//512):
        for w_idx in range(imgs.shape[1]//512):
            
            patch = imgs[h_idx*512:h_idx*512+512, w_idx*512:w_idx*512+512, :]
                
            PATH_NPY = f'{cfg.PATH_NPY_ROOT}img/{SCENE_ID}_h{h_idx}_w{w_idx}.npy'
            np.save(PATH_NPY, patch)
            PATHS_NPY.append(PATH_NPY)


  0%|          | 0/11 [00:00<?, ?it/s]

11-T-PH-2022-10-20


  9%|▉         | 1/11 [02:19<23:19, 139.96s/it]

11-T-PH-2022-10-15


 18%|█▊        | 2/11 [04:55<22:23, 149.29s/it]

...

 82%|████████▏ | 9/11 [25:54<06:19, 189.74s/it]

11-T-PH-2022-7-22


 91%|█████████ | 10/11 [28:39<03:02, 182.10s/it]

11-T-PH-2022-7-7


100%|██████████| 11/11 [31:18<00:00, 170.79s/it]


In [15]:
df = pd.DataFrame({
    'feature': PATHS_NPY
})

df['name'] = df['feature'].apply(lambda x: x.split('/')[-1].split('.')[0])
df['mask'] = False
df['defects'] = False
df['fold'] = -1
df['hight'] = 512
df['width'] = 512
df['california'] = False
df['cv'] = -1
df['area'] = 'Arizona'

df.head(4)

,feature,name,mask,defects,fold,hight,width,california,cv,area
0,../data/021_Arizona/img/11-T-PH-2022-10-20_h0_...,11-T-PH-2022-10-20_h0_w0,False,False,-1,512,512,False,-1,Arizona
1,../data/021_Arizona/img/11-T-PH-2022-10-20_h0_...,11-T-PH-2022-10-20_h0_w1,False,False,-1,512,512,False,-1,Arizona
2,../data/021_Arizona/img/11-T-PH-2022-10-20_h0_...,11-T-PH-2022-10-20_h0_w2,False,False,-1,512,512,False,-1,Arizona
3,../data/021_Arizona/img/11-T-PH-2022-10-20_h0_...,11-T-PH-2022-10-20_h0_w3,False,False,-1,512,512,False,-1,Arizona


In [16]:
df.to_csv(cfg.PATH_CSV, index=False, header=True)